In [3]:
import os
import time
import zipfile
import numpy as np
import random

from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import os
import time
import math



# Set global random seeds
random.seed(42)
np.random.seed(42)

# Set environment variables for deterministic behavior
os.environ['PYTHONHASHSEED'] = '42'
os.environ['OMP_NUM_THREADS'] = '1'  # For OpenMP
os.environ['MKL_NUM_THREADS'] = '1'  # For MKL
os.environ['OPENBLAS_NUM_THREADS'] = '1'  # For OpenBLAS
os.environ['NUMEXPR_NUM_THREADS'] = '1'  # For NumExpr
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'  # For macOS Accelerate

from sklearn.feature_extraction.text import CountVectorizer


def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        dirs.sort()  # Sort directories to ensure consistent traversal
        files.sort()  # Sort files to ensure consistent order
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")

    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")

    return dataPointsList

###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = 'Dataset/flaky_files.zip'
nonFlakyZip = 'Dataset/reduced_nonflaky_files.zip'
imbalanceNonFlakyZip = 'Dataset/nonflaky_files.zip'

# Create directories
outDirbalance = "results/balance_flaky_nonflaky/"
outDirimbalance = "results/imbalance_nonflaky/"
os.makedirs(outDirbalance, exist_ok=True)
os.makedirs(outDirimbalance, exist_ok=True)

extractDirbalance = "extracted/balance_flaky_nonflaky/"
extractDirimbalance = "extracted/imbalance_nonflaky/"
os.makedirs(extractDirbalance, exist_ok=True)
os.makedirs(extractDirimbalance, exist_ok=True)

# Extract and read data once for balance combination
flakyDirbalance = os.path.join(extractDirbalance, 'flaky')
nonFlakyDirbalance = os.path.join(extractDirbalance, 'nonFlaky')
os.makedirs(flakyDirbalance, exist_ok=True)
os.makedirs(nonFlakyDirbalance, exist_ok=True)

extract_zip(flakyZip, flakyDirbalance)
extract_zip(nonFlakyZip, nonFlakyDirbalance)

dataPointsFlakybalance = getDataPoints(flakyDirbalance)
dataPointsNonFlakybalance = getDataPoints(nonFlakyDirbalance)
dataPointsbalance = dataPointsFlakybalance + dataPointsNonFlakybalance

# Print the number of datasets for balance combination
print(f"Number of flaky documents (balance combination): {len(dataPointsFlakybalance)}")
print(f"Number of non-flaky documents (balance combination): {len(dataPointsNonFlakybalance)}")
print(f"Total number of documents (balance combination): {len(dataPointsbalance)}")

dataLabelsListbalance = np.array([1]*len(dataPointsFlakybalance) + [0]*len(dataPointsNonFlakybalance))

# Extract and read data once for imbalance non-flaky combination
flakyDirimbalance = os.path.join(extractDirimbalance, 'flaky')
nonFlakyDirimbalance = os.path.join(extractDirimbalance, 'nonFlaky')
os.makedirs(flakyDirimbalance, exist_ok=True)
os.makedirs(nonFlakyDirimbalance, exist_ok=True)

extract_zip(flakyZip, flakyDirimbalance)
extract_zip(imbalanceNonFlakyZip, nonFlakyDirimbalance)

dataPointsFlakyimbalance = getDataPoints(flakyDirimbalance)
dataPointsNonFlakyimbalance = getDataPoints(nonFlakyDirimbalance)
dataPointsimbalance = dataPointsFlakyimbalance + dataPointsNonFlakyimbalance

# Print the number of datasets for imbalance combination
print(f"Number of flaky documents (imbalance combination): {len(dataPointsFlakyimbalance)}")
print(f"Number of non-flaky documents (imbalance combination): {len(dataPointsNonFlakyimbalance)}")
print(f"Total number of documents (imbalance combination): {len(dataPointsimbalance)}")

dataLabelsListimbalance = np.array([1]*len(dataPointsFlakyimbalance) + [0]*len(dataPointsNonFlakyimbalance))


Number of flaky documents (balance combination): 45
Number of non-flaky documents (balance combination): 45
Total number of documents (balance combination): 90
Number of flaky documents (imbalance combination): 45
Number of non-flaky documents (imbalance combination): 243
Total number of documents (imbalance combination): 288


## KNN ##

In [4]:
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import os
import time
import math

# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)
def runKNN(dataPoints, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define the pipeline with CountVectorizer, PCA, and KNN
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),
        ('pca', PCA(random_state=42)),
        ('knn', KNeighborsClassifier())
    ])

    # Parameter grid for hyperparameter tuning
    if combination_label == "balance":
        param_grid = {
            'pca__n_components': [50, 60, 65],  # Number of components for PCA
            'knn__n_neighbors': [3, 5, 7, 9],
            'knn__metric': ['cosine', 'euclidean'],
            'knn__weights': ['uniform', 'distance'],
        }
    else:
        param_grid = {
            'pca__n_components': [150, 180, 200, 220],  # Number of components for PCA
            'knn__n_neighbors': [3, 5, 7, 9],
            'knn__metric': ['cosine', 'euclidean'],
            'knn__weights': ['uniform', 'distance'],
        }

    # Custom scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,  # MCC score custom function
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',  # Optimize the model based on F1 score
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(dataPoints, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    # Get the index of the best parameter combination
    best_index = grid_search.best_index_
    # Get the standard deviation of F1 score for the best parameter combination
    std_f1_best = grid_search.cv_results_['std_test_f1'][best_index]

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score} (Std Dev: {std_f1_best})")

    # Save the results
    outFile = f"{combination_label}-params-knn-{n_splits}-folds.csv"
    os.makedirs(outDir, exist_ok=True)
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Updated CSV header to include standard deviation columns
        fo.write("pca_n_components,n_neighbors,metric,weights,accuracy,std_accuracy,precision,std_precision,recall,std_recall,f1,std_f1,mcc,std_mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            std_accuracy = grid_search.cv_results_['std_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            std_precision = grid_search.cv_results_['std_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            std_recall = grid_search.cv_results_['std_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            std_f1 = grid_search.cv_results_['std_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            std_mcc = grid_search.cv_results_['std_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            # Write the parameters, mean scores, and standard deviations to the CSV
            fo.write(f"{param['pca__n_components']},{param['knn__n_neighbors']},{param['knn__metric']},{param['knn__weights']},"
                     f"{accuracy},{std_accuracy},{precision},{std_precision},{recall},{std_recall},{f1},{std_f1},{mcc},{std_mcc},{preparationTime}\n")

    print(f"KNN analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    # Return std_f1_best along with other results
    return best_params, best_score, std_f1_best

# Run KNN on balance combination
print("\nStarting KNN analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_knn_balance, best_score_5folds_knn_balance, std_f1_best_balance = runKNN(
    dataPointsbalance, dataLabelsListbalance, outDirbalance, 5, "balance"
)

# Display results
print("\nBest results for KNN 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_knn_balance}")
print(f"Best F1 Score: {best_score_5folds_knn_balance} (Std Dev: {std_f1_best_balance})")

# Run KNN on imbalance non-flaky combination
print("\nStarting KNN analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_knn_imbalance, best_score_5folds_knn_imbalance, std_f1_best_imbalance = runKNN(
    dataPointsimbalance, dataLabelsListimbalance, outDirimbalance, 5, "imbalance"
)

# Display results
print("\nBest results for KNN 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_knn_imbalance}")
print(f"Best F1 Score: {best_score_5folds_knn_imbalance} (Std Dev: {std_f1_best_imbalance})")





Starting KNN analysis for flaky vs smaller non-flaky files (balance combination)...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 3, 'knn__weights': 'distance', 'pca__n_components': 65}
Best F1 Score: 0.7158823529411764 (Std Dev: 0.11535412604004967)
KNN analysis completed for 5-folds. Results saved to: balance-params-knn-5-folds.csv

Best results for KNN 5-fold on balance combination:
Best Parameters: {'knn__metric': 'euclidean', 'knn__n_neighbors': 3, 'knn__weights': 'distance', 'pca__n_components': 65}
Best F1 Score: 0.7158823529411764 (Std Dev: 0.11535412604004967)

Starting KNN analysis for flaky vs imbalance non-flaky files (imbalance combination)...
Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best Parameters: {'knn__metric': 'cosine', 'knn__n_neighbors': 3, 'knn__weights': 'distance', 'pca__n_components': 150}
Best F1 Score: 0.6366946778711485 (Std Dev: 0.08242912690086895)
KNN a

## SVM

In [ ]:
import os
import time
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import make_scorer

# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)
def runSVM(dataPoints, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define the pipeline with CountVectorizer, PCA, and SVM
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),
        ('pca', PCA(random_state=42)),
        ('svm', SVC(probability=True, random_state=42))  # Enable probability estimates for threshold tuning
    ])

    # Parameter grid for hyperparameter tuning
    if combination_label == "balance":
        param_grid = {
            'pca__n_components': [50, 60, 65],  # Number of principal components
            'svm__C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
            'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel types
        }
    else:
        param_grid = {
            'pca__n_components': [150, 180, 200, 220],
            'svm__C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
            'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel types
        }

    # Custom scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,  # MCC score custom function
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(dataPoints, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    # Get the index of the best parameter combination
    best_index = grid_search.best_index_
    # Get the standard deviation of F1 score for the best parameter combination
    std_f1_best = grid_search.cv_results_['std_test_f1'][best_index]

    # Print best parameters and F1 score with standard deviation
    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score:.4f} (Std Dev: {std_f1_best:.4f})")

    # Save the results
    outFile = f"{combination_label}-params-svm-{n_splits}-folds.csv"
    os.makedirs(outDir, exist_ok=True)
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Updated CSV header to include standard deviation columns
        fo.write("pca_n_components,C,kernel,accuracy,std_accuracy,precision,std_precision,recall,std_recall,f1,std_f1,mcc,std_mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            std_accuracy = grid_search.cv_results_['std_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            std_precision = grid_search.cv_results_['std_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            std_recall = grid_search.cv_results_['std_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            std_f1 = grid_search.cv_results_['std_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            std_mcc = grid_search.cv_results_['std_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            # Write the parameters, mean scores, and standard deviations to the CSV
            fo.write(f"{param['pca__n_components']},{param['svm__C']},{param['svm__kernel']},"
                     f"{accuracy},{std_accuracy},{precision},{std_precision},{recall},{std_recall},"
                     f"{f1},{std_f1},{mcc},{std_mcc},{preparationTime}\n")

    print(f"SVM analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    # Return std_f1_best along with other results
    return best_params, best_score, std_f1_best
# Run SVM on balance combination
print("\nStarting SVM analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_svm_balance, best_score_5folds_svm_balance, std_f1_best_balance = runSVM(
    dataPointsbalance, dataLabelsListbalance, outDirbalance, 5, "balance"
)

# Display results
print("\nBest results for SVM 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_svm_balance}")
print(f"Best F1 Score: {best_score_5folds_svm_balance} (Std Dev: {std_f1_best_balance})")

# Run SVM on imbalance non-flaky combination
print("\nStarting SVM analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_svm_imbalance, best_score_5folds_svm_imbalance, std_f1_best_imbalance = runSVM(
    dataPointsimbalance, dataLabelsListimbalance, outDirimbalance, 5, "imbalance"
)

# Display results
print("\nBest results for SVM 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_svm_imbalance}")
print(f"Best F1 Score: {best_score_5folds_svm_imbalance} (Std Dev: {std_f1_best_imbalance})")



Starting SVM analysis for flaky vs smaller non-flaky files (balance combination)...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'pca__n_components': 50, 'svm__C': 100.0, 'svm__kernel': 'rbf'}
Best F1 Score: 0.8455 (Std Dev: 0.0948)
SVM analysis completed for 5-folds. Results saved to: balance-params-svm-5-folds.csv

Best results for SVM 5-fold on balance combination:
Best Parameters: {'pca__n_components': 50, 'svm__C': 100.0, 'svm__kernel': 'rbf'}
Best F1 Score: 0.845545235638115 (Std Dev: 0.09477558071599416)

Starting SVM analysis for flaky vs imbalance non-flaky files (imbalance combination)...
Fitting 5 folds for each of 80 candidates, totalling 400 fits


## XGB

In [3]:
import pandas as pd
from xgboost import XGBClassifier
import time
import os
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runXGB(dataPoints, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    print(f"Starting runXGB with combination_label: {combination_label}")

    # Define the pipeline with CountVectorizer, PCA, and XGBoost
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),
        ('pca', PCA(random_state=42)),
        ('xgb', XGBClassifier(eval_metric="logloss", use_label_encoder=False, random_state=42))
    ])

    # Parameter grid for hyperparameter tuning
    if combination_label == "balance":
        param_grid = {
            'pca__n_components': [50, 60, 65],
            'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5],
            'xgb__max_depth': [3, 5, 7, 10],
            'xgb__n_estimators': [50, 100, 200],
        }
    else:
        param_grid = {
            'pca__n_components': [150, 180, 200, 220],
            'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5],
            'xgb__max_depth': [3, 5, 7, 10],
            'xgb__n_estimators': [100, 150, 200],
        }

    print(f"Parameter grid for {combination_label}: {param_grid}")

    # Scoring metrics including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer,
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with the pipeline
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=skf,
        scoring=scoring,
        refit='f1',
        verbose=1,
        return_train_score=True
    )

    # Fit the GridSearchCV on data
    grid_search.fit(dataPoints, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    # Get the index of the best parameter combination
    best_index = grid_search.best_index_
    # Get the standard deviation of F1 score for the best parameter combination
    std_f1_best = grid_search.cv_results_['std_test_f1'][best_index]

    print(f"Best Parameters for {combination_label}: {best_params}")
    print(f"Best F1 Score for {combination_label}: {best_score:.4f} (Std Dev: {std_f1_best:.4f})")

    # Save the results
    outFile = f"{combination_label}-params-xgb-{n_splits}-folds.csv"
    output_path = os.path.join(outDir, outFile)
    print(f"Saving results to: {output_path}")

    with open(output_path, "w") as fo:
        # Updated CSV header to include standard deviation columns
        fo.write("pca_n_components,learning_rate,max_depth,n_estimators,accuracy,std_accuracy,precision,std_precision,recall,std_recall,f1,std_f1,mcc,std_mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            std_accuracy = grid_search.cv_results_['std_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            std_precision = grid_search.cv_results_['std_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            std_recall = grid_search.cv_results_['std_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            std_f1 = grid_search.cv_results_['std_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            std_mcc = grid_search.cv_results_['std_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)
            # Include pca__n_components in the output
            fo.write(f"{param['pca__n_components']},{param['xgb__learning_rate']},{param['xgb__max_depth']},{param['xgb__n_estimators']},"
                     f"{accuracy},{std_accuracy},{precision},{std_precision},{recall},{std_recall},{f1},{std_f1},{mcc},{std_mcc},{preparationTime}\n")

    print(f"XGBoost analysis completed for {n_splits}-folds on {combination_label} data. Results saved to: {outFile}")
    # Return std_f1_best along with other results
    return best_params, best_score, std_f1_best

# Run XGBoost on imbalance non-flaky combination
print("\nStarting XGBoost analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_xgb_imbalance, best_score_5folds_xgb_imbalance, std_f1_best_imbalance = runXGB(
    dataPointsimbalance, dataLabelsListimbalance, outDirimbalance, 5, "imbalance"
)

# Display results for imbalance combination
print("\nBest results for XGBoost 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_xgb_imbalance}")
print(f"Best F1 Score: {best_score_5folds_xgb_imbalance:.4f} (Std Dev: {std_f1_best_imbalance:.4f})")

# Run XGBoost on balance combination
print("\nStarting XGBoost analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_xgb_balance, best_score_5folds_xgb_balance, std_f1_best_balance = runXGB(
    dataPointsbalance, dataLabelsListbalance, outDirbalance, 5, "balance"
)

# Display results for balance combination
print("\nBest results for XGBoost 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_xgb_balance}")
print(f"Best F1 Score: {best_score_5folds_xgb_balance:.4f} (Std Dev: {std_f1_best_balance:.4f})")



Starting XGBoost analysis for flaky vs imbalance non-flaky files (imbalance combination)...
Starting runXGB with combination_label: imbalance
Parameter grid for imbalance: {'pca__n_components': [150, 180, 200, 220], 'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5], 'xgb__max_depth': [3, 5, 7, 10], 'xgb__n_estimators': [100, 150, 200]}
Fitting 5 folds for each of 192 candidates, totalling 960 fits


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:04:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:04:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:04:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are

Best Parameters for imbalance: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 7, 'xgb__n_estimators': 200}
Best F1 Score for imbalance: 0.6247 (Std Dev: 0.1969)
Saving results to: results/imbalance_nonflaky/imbalance-params-xgb-5-folds.csv
XGBoost analysis completed for 5-folds on imbalance data. Results saved to: imbalance-params-xgb-5-folds.csv

Best results for XGBoost 5-fold on imbalance combination:
Best Parameters: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 7, 'xgb__n_estimators': 200}
Best F1 Score: 0.6247 (Std Dev: 0.1969)

Starting XGBoost analysis for flaky vs smaller non-flaky files (balance combination)...
Starting runXGB with combination_label: balance
Parameter grid for balance: {'pca__n_components': [50, 60, 65], 'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5], 'xgb__max_depth': [3, 5, 7, 10], 'xgb__n_estimators': [50, 100, 200]}
Fitting 5 folds for each of 144 candidates, totalling 720 fits


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:19:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:19:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are

Best Parameters for balance: {'pca__n_components': 60, 'xgb__learning_rate': 0.3, 'xgb__max_depth': 3, 'xgb__n_estimators': 100}
Best F1 Score for balance: 0.8161 (Std Dev: 0.0965)
Saving results to: results/balance_flaky_nonflaky/balance-params-xgb-5-folds.csv
XGBoost analysis completed for 5-folds on balance data. Results saved to: balance-params-xgb-5-folds.csv

Best results for XGBoost 5-fold on balance combination:
Best Parameters: {'pca__n_components': 60, 'xgb__learning_rate': 0.3, 'xgb__max_depth': 3, 'xgb__n_estimators': 100}
Best F1 Score: 0.8161 (Std Dev: 0.0965)


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:22:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
